</div>
<div>
<img src="graphics/logo-banner-bw.png" width="500"/>
</div>
<div>
    <h2 style="text-align:center"> Sensitivity Analysis and Probabilistic Estimation of Reservoir Emissions</h2>
    <h3> This notebook demonstrates how to: </h3>
 <ol>
  <li>Run parametric uncertainty analysis with SALib and the Sobol method</li>
  <li>Visualize the parametric sensitivity / uncertainty on various plots</li>
  <li>Compute the sensitivities across many scenarios (e.g. reservoirs)</li>
  <li>Present emission predictions as probability density plots</li>
</ol> 
</div>

#### NOTE:
- Requires **SALib** library - Python implementations of commonly used sensitivity analysis methods, including Sobol, Morris, and FAST methods - see: https://app.readthedocs.org/projects/salib/downloads/pdf/stable/

<!--NAVIGATION-->
<font size="3"> < [Modifying Configuration Parametrers](05-Modifying-Configuration-Parameters.ipynb) | [Contents](index.ipynb) </font>

<a href="https://colab.research.google.com/github/tomjanus/re-emission/blob/master/docs/notebooks/06-Parametric-Uncertainty.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

#### QUICK DESCRIPTION:

In this example, we leverage Re-Emission's capability to dynamically alter its configuration parameters, such as emission model regression coefficients, pre-impoundment emissions or nutrient exports to perform sensitivity analysis and Monte-Carlo simulations of reservoir emissions under parametric uncertainties. The analysis is performed using Python's **SALib** package. ReEmission's `salib` module contains interfaces to **SALib** methods and functions allowing seamless integration with **SALib** enabling global sensitivity analysis to parametric uncertainties as well as input uncertainties.

This notebook performs a simple analysis to briefly demonstrate the capability of ReEmission's `salib` module. For simplicity, it is restricted to testing model sensitivity to parametric uncertainties stemming from emission regression equations only. Sensitivity to other parametric uncertainties and to input uncertainties have not been investigated here.

In [ ]:
import pathlib
import os
from typing import List
from functools import partial
import gdown
from rich import print as rprint
%matplotlib inline
import matplotlib.pyplot as plt
try:
    import reemission
except ImportError:
    print("Unable to import reemission. Please ensure it is installed.")
    %pip install git+https://github.com/tomjanus/reemission.git --quiet
from reemission.salib.runners import SALibProblem, SobolAnalyser # type: ignore
from reemission.salib.wrappers import ReEmissionSALibWrapper # type: ignore
from reemission.salib.visualize import SobolResultVisualizer, SobolScenarioResultsVisualizer # type: ignore
from reemission.salib.specloaders import (
    ReEmissionSALibSpecLoader,
    set_unit_input_distribution_using_rel_diffrence)  # type: ignore
from reemission.input import Inputs  # type: ignore
from reemission.salib.runners import SobolResults, SobolScenarioResults  # type: ignore

# Constants
REL_DIFF = 0.1

# Get the directory where this notebook is located
notebook_dir = pathlib.Path().resolve()

# Define file paths relative to notebook location
inputs_file = notebook_dir / 'inputs_sensitivity_test.json'
spec_file = notebook_dir / 'uncertain_parameter_specification.yaml'

if not inputs_file.exists():
    # Download the required input file from an external link
    !gdown 1YHjA9HfulLV6wqwzryHZiFFc1kvXKFWL -O {str(inputs_file)}
    
if not spec_file.exists():
    # Download the required input file from an external link
    !gdown 1punHksgsZhj7Tq7IPPLCLqa0AHkDrLmx -O {str(spec_file)}

### 2. Sensitivity Analysis using Re-Emission

* Confidence intervals for some parameters were derived from [G-Res Tool Technical Documentation](https://www.hydropower.org/publications/the-ghg-reservoir-tool-g-res-technical-documentation)

In [ ]:

sc_results: List[SobolResults] = []
sc_names: List[str] = []
seed = 42
selected_reservoir_index = 1
# Run SOBOL analysis for a subset of UK reservoirs - can take a bit of time
inputs = Inputs.fromfile("inputs_sensitivity_test.json")
reservoirs_list_uk = list(inputs.inputs.keys())
rprint(f"Number of reservoirs: {len(reservoirs_list_uk)}")
selected_reservoirs = reservoirs_list_uk[:]
uk_input_file = pathlib.Path("inputs_sensitivity_test.json").resolve()
spec_file = "uncertain_parameter_specification.yaml"
inputs = Inputs.fromfile(uk_input_file)

for res_no, reservoir in enumerate(selected_reservoirs):
    rprint(f"Running SOBOL analysis for reservoir: {reservoir} - {res_no + 1} out of {len(selected_reservoirs)}")
    selected_input = inputs.get_input(reservoir) # Use a single reservoir
    # Set the relative +/- difference for the inputs with missing distributions
    rel_difference: float = 0.1
    # Load the SALib specification for the re-emission model
    reemission_salib_spec = ReEmissionSALibSpecLoader(
        spec_file=spec_file,
        input=selected_input,
        missing_input_dist_handler = 
            partial(
                set_unit_input_distribution_using_rel_diffrence,
                rel_difference=rel_difference)
    )
    # Var names for visualization
    var_names = reemission_salib_spec.var_name_map
    # Create a list of variables from the SALib specification
    reemission_variables = reemission_salib_spec.list_of_variables
    # Create a list of accessors from the SALib specification
    accessors = reemission_salib_spec.accessors
    reemission_salib_problem = SALibProblem.from_variables(
        reemission_variables
    )
    reemission_salib_model = ReEmissionSALibWrapper.from_variables(
        variables = reemission_variables,
        input = selected_input,
        emission = 'total_net',
        accessors = accessors
    )
    analyser = SobolAnalyser(
        problem = reemission_salib_problem,
        variables = reemission_variables,
        model = reemission_salib_model,
        num_samples = 512
    )
    sc_result = analyser.run_sobol()
    object.__setattr__(sc_result, 'nominal_output', sc_result.nominal_output[0])
    sc_results.append(sc_result)
    sc_names.append(reservoir)
    
sc_results = SobolScenarioResults(
    sc_names=sc_names,
    results=sc_results
)
var_names = sc_results.var_names
visualizer = SobolResultVisualizer(
    sc_results.results[selected_reservoir_index],
    par_name_map = var_names)
scenario_visualizer = SobolScenarioResultsVisualizer(sc_results)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()
scenario_visualizer.plot_S1_ST(
    ax=axes[0],
    x_label_rotation = 90,
    title=f'Sobol Indices for regression coefficients')

scenario_visualizer.plot_variance_contributions_by_group(
    ax=axes[1],
    title="Variance contributions by uncertainty group",)

scenario_visualizer.plot_outputs_per_scenarios(
    ax=axes[2], 
    #scenario_names = [f'{ix}' for ix in range(len(sc_results.scenario_names))],
    x_label_rotation = 90,
    sorting='desc',
    title="Total net emission predictions for multiple reservoirs",
    width=0.085,
    component_colors=['#ff7f0e', '#1f77b4', '#2ca02c', '#9467bd', '#d62728', '#8c564b'])

visualizer.plot_output_kde(
    ax=axes[3],
    xlims=(-1400,-1000),
    title=f"Total net emissions under parameter uncertainty - {sc_names[selected_reservoir_index]}")

fig.savefig(pathlib.Path('reemission_sobol_paper.png'))

<a href="https://colab.research.google.com/github/tomjanus/re-emission/blob/master/docs/notebooks/06-Parametric-Uncertainty.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>